In [2]:
# Import Libraries

In [3]:
# import libraries
import numpy as np
import pandas as pd
import os

# 1 Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise. 

In [6]:
# import data
path = r'/Users/oliviamurphy/Desktop/2022-01 Insta Cart Analysis/02 Data'
orders_products_departments_final = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_departments_final_new_variables.pkl'))

In [7]:
orders_products_departments_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32433030 entries, 0 to 32433029
Data columns (total 26 columns):
 #   Column                        Dtype   
---  ------                        -----   
 0   product_id                    string  
 1   product_name                  object  
 2   aisle_id                      string  
 3   department_id                 string  
 4   prices                        float64 
 5   order_id                      string  
 6   user_id                       string  
 7   order_number                  int64   
 8   orders_day_of_week            int64   
 9   order_hour_of_day             int64   
 10  days_passed_since_last_order  float64 
 11  add_to_cart_order             int64   
 12  reordered                     int64   
 13  gender                        object  
 14  state                         object  
 15  age                           int64   
 16  date_joined                   object  
 17  number_of_dependants          int64   
 18  

# 2 In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [9]:
#aggregate data
orders_products_departments_final.groupby('department_id').agg({'order_number': ["mean"]})

,order_number
,mean
department_id,
1,15.457687
10,20.197148
11,16.170371
12,15.887622
13,16.583304
14,16.759763
15,16.165037
16,17.663250


#  3 - Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

# Loyalty Flag

In [10]:
#finds max number of orders
orders_products_departments_final['max_orders'] = orders_products_departments_final.groupby(['user_id'])['order_number'].transform(np.max)

In [11]:
#creates flag for Loyal customer
orders_products_departments_final.loc[orders_products_departments_final['max_orders']>40, 'loyalty_flag'] = 'Loyal customer'

In [12]:
#creates flag for Regular customer
orders_products_departments_final.loc[(orders_products_departments_final['max_orders']<=40) & (orders_products_departments_final['max_orders']>10), 'loyalty_flag'] = 'Regular customer'
                                                             
                                                             

In [13]:
#creates flag for New Customer
orders_products_departments_final.loc[orders_products_departments_final['max_orders']<10, 'loyalty_flag'] = 'New customer'

In [14]:
#check counts
orders_products_departments_final['loyalty_flag'].value_counts(dropna = False)

Regular customer    15890458
Loyal customer      10293434
New customer         5475359
NaN                   773779
Name: loyalty_flag, dtype: int64

# 5 The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [15]:
# spending habits 
orders_products_departments_final.groupby('loyalty_flag').agg({'prices':['mean', 'min', 'max']})


prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.778465  1.0  25.0
New customer      7.804473  1.0  25.0
Regular customer  7.803009  1.0  25.0

# Loyal customers spend less than new customers on average on products. Regular customers spend a little less than new customers on a product (on average)

# 6 The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
## If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
## If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.

In [16]:
#Creates average price of product for each user
orders_products_departments_final['average_spend'] = orders_products_departments_final.groupby('user_id')['prices'].transform(np.average)

In [17]:
#checks output of average spend
orders_products_departments_final['average_spend'].head()

0    6.935811
1    6.935811
2    6.935811
3    6.935811
4    6.935811
Name: average_spend, dtype: float64

In [18]:
#Defines low spender
orders_products_departments_final.loc[orders_products_departments_final['average_spend']<10, 'spender_type'] = 'Low spender'

In [19]:
#Defines high spender
orders_products_departments_final.loc[orders_products_departments_final['average_spend']>=10, 'spender_type'] = 'High spender'

In [20]:
#Checks counts
orders_products_departments_final['spender_type'].value_counts(dropna = False)

Low spender     31797621
NaN               515447
High spender      119962
Name: spender_type, dtype: int64

# 7 In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
## If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
## If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
## If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [22]:
#find the median
orders_products_departments_final['median_days_since_prior_order'] = orders_products_departments_final.groupby('user_id')['days_passed_since_last_order'].transform(np.median)

In [23]:
#define nonfrequent customers
orders_products_departments_final.loc[orders_products_departments_final['median_days_since_prior_order'] > 20, 'Frequent_customer?'] = "Non-Frequent customer"

In [24]:
#define frequent customers
orders_products_departments_final.loc[orders_products_departments_final['median_days_since_prior_order'] <= 10, 'Frequent_customer?'] = "Frequent customer"

In [25]:
#define regular customers
orders_products_departments_final.loc[(orders_products_departments_final['median_days_since_prior_order'] <= 20) & (orders_products_departments_final['median_days_since_prior_order'] > 10), 'Frequent_customer?'] = "Regular customer"

In [26]:
#check value counts
orders_products_departments_final['Frequent_customer?'].value_counts(dropna = False)

Frequent customer        21576963
Regular customer          7216644
Non-Frequent customer     3639423
Name: Frequent_customer?, dtype: int64

In [27]:
orders_products_departments_final.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,price_range_loc,busiest day,busiest_days,busiest_period_of_day,max_orders,loyalty_flag,average_spend,spender_type,median_days_since_prior_order,Frequent_customer?
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Mid range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Mid range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,4913,Table Water Crackers,78,19,4.4,894221,138,23,5,13,...,Low-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
3,11759,Organic Simply Naked Pita Chips,107,19,4.4,1986630,138,7,0,12,...,Low-range product,Busiest day,Busiest day,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
4,13424,Almonds,45,19,4.2,3139998,138,28,6,11,...,Low-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer


# 9 Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder

In [28]:
#export file
orders_products_departments_final.to_pickle(os.path.join(path, 'Prepared Data', 'orders_products_departments_final_agg.pkl'))